In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from prophet import Prophet
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')  # Prophet can be verbose with warnings

class RideDataset(Dataset):
    def __init__(self, features, ride_indices, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.ride_indices = torch.tensor(ride_indices, dtype=torch.long)
        self.targets = torch.tensor(targets, dtype=torch.float32).reshape(-1, 1)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.ride_indices[idx], self.targets[idx]

class ResidualPredictor(nn.Module):
    def __init__(self, input_dim, num_rides, hidden_dims=[64, 32]):
        super(ResidualPredictor, self).__init__()
        self.ride_embedding = nn.Embedding(num_rides, 8)  # 8-dimensional ride embedding
        
        layers = []
        # Input layer (features + ride embedding)
        layers.append(nn.Linear(input_dim + 8, hidden_dims[0]))
        layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(hidden_dims[0]))
        layers.append(nn.Dropout(0.2))
        
        # Hidden layers
        for i in range(len(hidden_dims)-1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_dims[i+1]))
            layers.append(nn.Dropout(0.2))
            
        # Output layer
        layers.append(nn.Linear(hidden_dims[-1], 1))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x, ride_idx):
        ride_emb = self.ride_embedding(ride_idx)
        combined = torch.cat([x, ride_emb], dim=1)
        return self.model(combined)

def extract_time_features(df):
    """Extract cyclical time features from timestamp"""
    # Hour of day (0-23)
    df['hour_sin'] = np.sin(2 * np.pi * df['timestamp'].dt.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['timestamp'].dt.hour / 24)
    
    # Day of week (0-6)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['timestamp'].dt.dayofweek / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['timestamp'].dt.dayofweek / 7)
    
    # Month (1-12)
    df['month_sin'] = np.sin(2 * np.pi * df['timestamp'].dt.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['timestamp'].dt.month / 12)
    
    # Day of year (1-366)
    df['day_of_year_sin'] = np.sin(2 * np.pi * df['timestamp'].dt.dayofyear / 366)
    df['day_of_year_cos'] = np.cos(2 * np.pi * df['timestamp'].dt.dayofyear / 366)
    
    return df

def decompose_time_series(df, val_cutoff_date, test_cutoff_date):
    """
    Decompose time series using Prophet for all rides
    """
    # Get unique rides
    rides = df['ride_name'].unique()
    
    # Create dataframe to store results
    result_df = pd.DataFrame()
    
    # Store Prophet models
    prophet_models = {}
    
    for ride in tqdm(rides, desc="Fitting Prophet models"):
        # Filter data for this ride
        ride_data = df[df['ride_name'] == ride].copy()
        
        # Prepare data for Prophet
        prophet_df = ride_data.rename(columns={'timestamp': 'ds', 'wait_time': 'y'})
        prophet_df = prophet_df.dropna(subset=['y'])
        
        # Split into train and validation
        train_prophet = prophet_df[prophet_df['ds'] < val_cutoff_date]
        
        # Fit Prophet model on training data
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=True,
            seasonality_mode='multiplicative'  # Often works better for wait times
        )
        
        # Add holiday effects if they seem important
        if ride_data['is_german_holiday'].sum() > 0:
            model.add_country_holidays(country_name='DE')
            
        model.fit(train_prophet)
        prophet_models[ride] = model
        
        # Predict for all timestamps
        future = pd.DataFrame(df['timestamp'].unique(), columns=['ds'])
        forecast = model.predict(future)
        
        # Merge forecasts back to original data
        ride_data = pd.merge(
            ride_data, 
            forecast[['ds', 'trend', 'yearly', 'weekly', 'daily', 'yhat']], 
            left_on='timestamp', 
            right_on='ds', 
            how='left'
        )
        
        # Calculate residuals where we have actual wait times
        ride_data['residual'] = ride_data['wait_time'] - ride_data['yhat']
        ride_data['baseline'] = ride_data['yhat']
        
        # Add to result dataframe
        result_df = pd.concat([result_df, ride_data])
    
    return result_df, prophet_models

def prepare_features(df):
    """
    Prepare features for neural network
    """
    # Numerical features
    numerical_features = ['temperature', 'rain', 'wind', 
                         'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos',
                         'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos']
    
    # Boolean features (convert to int)
    boolean_features = ['is_german_holiday', 'is_swiss_holiday', 'is_french_holiday', 'closed']
    
    # Combine features
    feature_df = df[numerical_features + boolean_features].copy()
    
    # Convert boolean to int
    for col in boolean_features:
        feature_df[col] = feature_df[col].astype(int)
    
    # Handle missing values
    feature_df = feature_df.fillna(0)
    
    # Get ride indices (for embedding)
    rides = df['ride_name'].unique()
    ride_to_idx = {ride: idx for idx, ride in enumerate(rides)}
    ride_indices = df['ride_name'].map(ride_to_idx).values
    
    # Normalize numerical features
    scaler = StandardScaler()
    feature_array = scaler.fit_transform(feature_df.values)
    
    return feature_array, ride_indices, ride_to_idx, scaler

def train_neural_network(train_features, train_ride_indices, train_residuals,
                         val_features, val_ride_indices, val_residuals,
                         num_rides, input_dim):
    """
    Train neural network to predict residuals
    """
    # Create datasets
    train_dataset = RideDataset(train_features, train_ride_indices, train_residuals)
    val_dataset = RideDataset(val_features, val_ride_indices, val_residuals)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    
    # Initialize model
    model = ResidualPredictor(input_dim, num_rides, hidden_dims=[128, 64, 32])
    
    # Define loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)
    
    # Training loop
    num_epochs = 50
    best_val_loss = float('inf')
    best_model = None
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        
        for batch_features, batch_rides, batch_residuals in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features, batch_rides)
            loss = criterion(outputs, batch_residuals)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch_features, batch_rides, batch_residuals in val_loader:
                outputs = model(batch_features, batch_rides)
                loss = criterion(outputs, batch_residuals)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        
        # Learning rate scheduler
        scheduler.step(val_loss)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict().copy()
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Load best model
    model.load_state_dict(best_model)
    
    return model

def predict_wait_times(df, prophet_models, nn_model, ride_to_idx, scaler):
    """
    Make predictions by combining Prophet and neural network
    """
    # Get features
    features = df[['temperature', 'rain', 'wind', 
                  'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos',
                  'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos',
                  'is_german_holiday', 'is_swiss_holiday', 'is_french_holiday', 'closed']].copy()
    
    # Convert boolean to int
    for col in ['is_german_holiday', 'is_swiss_holiday', 'is_french_holiday', 'closed']:
        features[col] = features[col].astype(int)
    
    # Handle missing values
    features = features.fillna(0)
    
    # Scale features
    scaled_features = scaler.transform(features.values)
    
    # Get ride indices
    ride_indices = df['ride_name'].map(ride_to_idx).values
    
    # Convert to tensors
    features_tensor = torch.tensor(scaled_features, dtype=torch.float32)
    ride_indices_tensor = torch.tensor(ride_indices, dtype=torch.long)
    
    # Predict residuals
    nn_model.eval()
    with torch.no_grad():
        residuals = nn_model(features_tensor, ride_indices_tensor).numpy().flatten()
    
    # Add residuals to baseline prediction
    df['predicted_residual'] = residuals
    df['predicted_wait_time'] = df['baseline'] + df['predicted_residual']
    
    # Ensure non-negative wait times
    df['predicted_wait_time'] = df['predicted_wait_time'].clip(lower=0)
    
    return df

def evaluate_model(df):
    """
    Evaluate model performance
    """
    # Filter to rows with actual wait times
    eval_df = df.dropna(subset=['wait_time'])
    
    # Calculate errors
    eval_df['error'] = eval_df['predicted_wait_time'] - eval_df['wait_time']
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(eval_df['wait_time'], eval_df['predicted_wait_time']))
    mae = mean_absolute_error(eval_df['wait_time'], eval_df['predicted_wait_time'])
    
    print(f'RMSE: {rmse:.4f}')
    print(f'MAE: {mae:.4f}')
    
    # Calculate metrics by ride
    ride_metrics = eval_df.groupby('ride_name').apply(
        lambda x: pd.Series({
            'RMSE': np.sqrt(mean_squared_error(x['wait_time'], x['predicted_wait_time'])),
            'MAE': mean_absolute_error(x['wait_time'], x['predicted_wait_time']),
            'Count': len(x)
        })
    )
    
    return rmse, mae, ride_metrics

def plot_predictions(df, ride_name, start_date, end_date):
    """
    Plot actual vs predicted wait times for a specific ride and time period
    """
    ride_df = df[(df['ride_name'] == ride_name) & 
                (df['timestamp'] >= start_date) & 
                (df['timestamp'] <= end_date)]
    
    plt.figure(figsize=(12, 6))
    plt.plot(ride_df['timestamp'], ride_df['wait_time'], 'o-', label='Actual')
    plt.plot(ride_df['timestamp'], ride_df['baseline'], '--', label='Prophet Baseline')
    plt.plot(ride_df['timestamp'], ride_df['predicted_wait_time'], 'r-', label='Final Prediction')
    plt.title(f'Wait Time Predictions for {ride_name}')
    plt.xlabel('Date')
    plt.ylabel('Wait Time (minutes)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Main execution function
def run_wait_time_prediction(df, val_cutoff_date='2023-01-01', test_cutoff_date='2024-01-01'):
    """
    Run the full wait time prediction pipeline
    """
    # Convert dates if needed
    if isinstance(val_cutoff_date, str):
        val_cutoff_date = pd.to_datetime(val_cutoff_date)
    if isinstance(test_cutoff_date, str):
        test_cutoff_date = pd.to_datetime(test_cutoff_date)
    
    # Make sure timestamp is datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Extract time features
    print("Extracting time features...")
    df = extract_time_features(df)
    
    # Split data
    train_data = df[df['timestamp'] < val_cutoff_date]
    val_data = df[(df['timestamp'] >= val_cutoff_date) & (df['timestamp'] < test_cutoff_date)]
    test_data = df[df['timestamp'] >= test_cutoff_date]
    
    print(f"Train data: {len(train_data)} rows ({train_data['timestamp'].min()} to {train_data['timestamp'].max()})")
    print(f"Validation data: {len(val_data)} rows ({val_data['timestamp'].min()} to {val_data['timestamp'].max()})")
    print(f"Test data: {len(test_data)} rows ({test_data['timestamp'].min()} to {test_data['timestamp'].max()})")
    
    # Decompose time series
    print("Decomposing time series with Prophet...")
    decomposed_df, prophet_models = decompose_time_series(df, val_cutoff_date, test_cutoff_date)
    
    # Prepare features
    print("Preparing features...")
    feature_array, ride_indices, ride_to_idx, scaler = prepare_features(decomposed_df)
    
    # Split into train/val/test sets
    train_mask = decomposed_df['timestamp'] < val_cutoff_date
    val_mask = (decomposed_df['timestamp'] >= val_cutoff_date) & (decomposed_df['timestamp'] < test_cutoff_date)
    test_mask = decomposed_df['timestamp'] >= test_cutoff_date
    
    # Filter to rows with actual wait times for training
    train_has_wait = ~decomposed_df.loc[train_mask, 'wait_time'].isna()
    val_has_wait = ~decomposed_df.loc[val_mask, 'wait_time'].isna()
    
    train_features = feature_array[train_mask][train_has_wait]
    train_ride_indices = ride_indices[train_mask][train_has_wait]
    train_residuals = decomposed_df.loc[train_mask, 'residual'].dropna().values
    
    val_features = feature_array[val_mask][val_has_wait]
    val_ride_indices = ride_indices[val_mask][val_has_wait]
    val_residuals = decomposed_df.loc[val_mask, 'residual'].dropna().values
    
    num_rides = len(decomposed_df['ride_name'].unique())
    input_dim = train_features.shape[1]
    
    # Train neural network
    print("Training neural network...")
    nn_model = train_neural_network(
        train_features, train_ride_indices, train_residuals,
        val_features, val_ride_indices, val_residuals,
        num_rides, input_dim
    )
    
    # Make predictions
    print("Making predictions...")
    train_predictions = predict_wait_times(
        decomposed_df[train_mask], prophet_models, nn_model, ride_to_idx, scaler
    )
    val_predictions = predict_wait_times(
        decomposed_df[val_mask], prophet_models, nn_model, ride_to_idx, scaler
    )
    test_predictions = predict_wait_times(
        decomposed_df[test_mask], prophet_models, nn_model, ride_to_idx, scaler
    )
    
    # Combine predictions
    predictions_df = pd.concat([train_predictions, val_predictions, test_predictions])
    
    # Evaluate model
    print("\nTraining Set Performance:")
    train_rmse, train_mae, train_ride_metrics = evaluate_model(train_predictions)
    
    print("\nValidation Set Performance:")
    val_rmse, val_mae, val_ride_metrics = evaluate_model(val_predictions)
    
    print("\nTest Set Performance:")
    test_rmse, test_mae, test_ride_metrics = evaluate_model(test_predictions)
    
    return predictions_df, prophet_models, nn_model, ride_to_idx, scaler

# Example usage
if __name__ == "__main__":
    # Load your data
    # df = pd.read_csv('amusement_park_data.csv')
    
    # Run prediction pipeline
    # predictions_df, prophet_models, nn_model, ride_to_idx, scaler = run_wait_time_prediction(df)
    
    # Plot predictions for a specific ride
    # plot_predictions(predictions_df, 'alpine express enzian', '2023-07-01', '2023-07-07')
    
    print("This script provides a complete implementation of the Prophet + PyTorch approach")
    print("for predicting amusement park wait times.")
    print("\nTo use this code, you would need to:")
    print("1. Load your dataset")
    print("2. Call run_wait_time_prediction() with appropriate date cutoffs")
    print("3. Analyze the results using the evaluate_model() and plot_predictions() functions")